#### Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!



Prerequisites:

1. Create and OpenAI apikey from https://platform.openai.com/
2. store the apikey under .env
3. create ba python virtual environment 

In [60]:
# Importing the required libraries

import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
# Loading the environment variable
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")

In [4]:
# Checking the API_KEY

if not api_key:
    print("No API Key found- Please generate an api_key and store in .env")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("There are spaces at the start or end of the API key, please resolve")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [61]:
# Initialising OpenAI
openai=OpenAI()

In [8]:
# Calling out openai for preview
message="Hello GPT this is Vaman, just saying HI to you"
response=openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content":message}])


In [21]:
print(response.choices[0].message.content)

Hello Vaman! It's great to hear from you. How can I assist you today?


In [37]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    url: str
    title:str
    text: str

    def __init__(self, url):
        self.url=url
        response=requests.get(url, headers=headers)
        soup=BeautifulSoup(response.content, 'html.parser')
        self.title=soup.title.string if soup.title else "No title Found"
        for irrelevant in soup.body(["scrtipt", "style", "img", "input"]):
            irrelevant.decompose()
        self.text=soup.body.get_text(separator='\n', strip=True) 
        
        
        

In [38]:
web=Website("https://htmlcolorcodes.com/")

In [24]:
print(web.title)

HTML Color Codes


In [25]:
print(web.text)

HTML Color Codes
Picker
Convert
RGB to Hex
Hex to RGB
Chart
Names
HTML Color Names
Minecraft Color Codes
Bukkit Color Codes
Roblox Color Codes
Library
Tutorials
Resources
Font Generator
Picker
Convert
RGB to Hex
Hex to RGB
Chart
Names
HTML
Minecraft
Bukkit
Roblox
Library
Tutorials
Resources
Dixon & Moe
Email
HTML Color Codes
Get HTML color codes, Hex color codes, RGB and HSL values with our color picker, color chart and HTML color names. Let's go!
HEX
#FF5733
RGB
255, 87, 51
HSL
11, 80%, 60%
Export palette
Color Picker
Find that perfect color with our
color picker
and discover beautiful color harmonies, tints, shades and tones; input Hex color codes, RGB and HSL values, and generate HTML, CSS and SCSS styles.
Take it for a spin!
Color Chart
Looking for some already great color combinations? Our
color chart
features flat design colors, Google's Material design scheme and the classic web safe color palette, all with Hex color codes.
See all the charts!
IndianRed
#CD5C5C
rgb(205, 92, 92)


In [26]:
# defining system_prompt

system_prompt="You are an assistant that analyzes contents of a website; \
and privides a summary, ignore the content related to navigation, \
provide responses in the markdown"

In [34]:
def user_prompt(website):
    user_prompt=f"You are exploring a website titled {website.title}"
    user_prompt+=" please provide a short summary of the website\
    If website includes news or announcements, summarize that too \n\n "
    user_prompt+=website.text
    return user_prompt

In [27]:
system_prompt

'You are an assistant that analyzes contents of a website; and privides a summary, ignore the content related to navigation, provide responses in the markdown'

In [ ]:
def messages_for(website):
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content":user_prompt(website)}
    ]

In [52]:
def messages_for(website):
   return[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content":user_prompt(website)}
    ]
    
    

In [53]:
messages_for(web)

[{'role': 'system',
  'content': 'You are an assistant that analyzes contents of a website; and privides a summary, ignore the content related to navigation, provide responses in the markdown'},
 {'role': 'user',
  'content': "You are exploring a website titled HTML Color Codes please provide a short summary of the website    If website includes news or announcements, summarize that too \n\n HTML Color Codes\nPicker\nConvert\nRGB to Hex\nHex to RGB\nChart\nNames\nHTML Color Names\nMinecraft Color Codes\nBukkit Color Codes\nRoblox Color Codes\nLibrary\nTutorials\nResources\nFont Generator\nPicker\nConvert\nRGB to Hex\nHex to RGB\nChart\nNames\nHTML\nMinecraft\nBukkit\nRoblox\nLibrary\nTutorials\nResources\nDixon & Moe\nEmail\nHTML Color Codes\nGet HTML color codes, Hex color codes, RGB and HSL values with our color picker, color chart and HTML color names. Let's go!\nHEX\n#FF5733\nRGB\n255, 87, 51\nHSL\n11, 80%, 60%\nExport palette\nColor Picker\nFind that perfect color with our\ncolor 

In [62]:
# Function to pass the url to OpenAi and retrieve responses.

def summarize(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [55]:
summarize("https://htmlcolorcodes.com/")

'## Summary of HTML Color Codes Website\n\nThe **HTML Color Codes** website offers a comprehensive suite of tools and resources for web designers and developers focused on color management. Key features include:\n\n- **Color Picker**: Users can select colors and generate HTML, CSS, or SCSS styles from different formats such as Hex, RGB, and HSL.\n- **Color Chart**: A visual display of various color combinations, including flat design colors, Google’s Material design palette, and classic web-safe colors.\n- **HTML Color Names**: A guide providing a quick reference to over 140 named colors in HTML.\n- **Color Library**: A collection of more than 100 shades for inspiration in design projects.\n- **Tutorials**: Educational resources covering basic to advanced HTML and CSS techniques related to color.\n\nThe website aims to assist users, whether novices or experienced professionals, in mastering color codes, enhancing design accuracy and creativity.\n\n### News or Announcements\nThe content

In [56]:
# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))
    

In [59]:
display_summary("https://htmlcolorcodes.com/")

# HTML Color Codes Summary

HTML Color Codes is a comprehensive resource for finding and utilizing color codes in web design. It offers several tools and features:

- **Color Picker**: Users can select colors and explore various tonalities and harmonies, while generating corresponding Hex, RGB, and HSL values.
- **Color Charts**: The website includes a chart featuring flat design colors, Google's Material Design, and the traditional web safe palette.
- **HTML Color Names**: A guide is provided for quick reference to over 140 HTML color names, grouped by color.
- **Color Library**: An extensive library consists of over 100 shades, providing inspiration for design projects.
- **Tutorials**: It features tutorials which cater to beginners and advanced users covering HTML, CSS, and SCSS.
- **Resources**: A collection of helpful tools, such as color palette generators and stock photo sites.

The site emphasizes the usability of various color codes in web design, explaining the basics of Hex codes, RGB, and HSL notations, while allowing users to export their chosen palettes. 

Overall, HTML Color Codes serves as a valuable tool for both novice and experienced web designers seeking to effectively use color in their projects.

#### Extending the work to read job descriptions and highlight the key technologies

In [63]:
# Step 1: Creating prompts

system_prompt=" You are an assistant who is an expert in reading Job descriptions;\
please help to listout the key technologies from the Job description provided"

In [81]:
JD='''Job Description of Python Fullstack LLM Langchain Developer 
Title: Python Fullstack LLM Langchain Developer
Location: Bangalore/Hyderabad/Pune
Job Description:

Position/ TITLE: Senior Python and LLM Developer

Location: Offshore – Hyderabad/Bangalore/Pune
Who are we looking For:
We are on the lookout for a Python Developer with specialized skills in AI, ML, and NLP to enhance our team's capabilities. The ideal candidate will
minimum 6-8 years of experience deeply entrenched in Python development and possess a strong foundation in artificial intelligence, machine
learning, and natural language processing techniques. This role involves designing and implementing sophisticated software solutions that are
efficient, scalable, and lead the way in innovation.
Technical Skills – Must Have
• Develop and deploy advanced AI/ML models and NLP algorithms within our software solutions, using Python and Langchain.
• Collaborate with cross-functional teams to understand project requirements, design AI/ML-driven features, and integrate NLP capabilities into
our products.
• Maintain and improve the performance of existing AI/ML models and NLP solutions.
• Work closely with data scientists and engineers to leverage data, identify new opportunities for AI/ML applications, and enhance natural
language understanding and processing capabilities.
• Design, develop, and maintain robust and scalable back-end APIs using Python and relevant frameworks.
• Integrate with various databases and third-party APIs for data access and functionality.
• Ensure code quality, sustainability, and scalability by adhering to best practices and participating in code reviews.
• Contribute to CI/CD pipelines for automated deployments and testing using Kubernetes.
• Exposure to GCP - AI services (vertex AI, etc.), Azure - AI services, Aws- AI services, MLops, NLP.
• Stay abreast of developments in AI, ML, and NLP fields to ensure our solutions remain cutting-edge.
Technical Skills - Good to have
• Knowledge of additional languages like Java, C#, or Javascript is a plus.
Preferred Qualifications:
• Bachelors or Masters degree in Computer Science, Artificial Intelligence, or a related field.
• A portfolio showcasing AI/ML and NLP projects'''

In [70]:
def user_prompt(text):
    user_prompt=f"You are analyzing a job description: {text}"
    user_prompt+= "Read the job description and highlight the key technical skiils required \n"
    user_prompt+=text

    return user_prompt
    

In [82]:
user_prompt(JD)

"You are analyzing a job description: Job Description of Python Fullstack LLM Langchain Developer \nTitle: Python Fullstack LLM Langchain Developer\nLocation: Bangalore/Hyderabad/Pune\nJob Description:\n\nPosition/ TITLE: Senior Python and LLM Developer\n\nLocation: Offshore – Hyderabad/Bangalore/Pune\nWho are we looking For:\nWe are on the lookout for a Python Developer with specialized skills in AI, ML, and NLP to enhance our team's capabilities. The ideal candidate will\nminimum 6-8 years of experience deeply entrenched in Python development and possess a strong foundation in artificial intelligence, machine\nlearning, and natural language processing techniques. This role involves designing and implementing sophisticated software solutions that are\nefficient, scalable, and lead the way in innovation.\nTechnical Skills – Must Have\n• Develop and deploy advanced AI/ML models and NLP algorithms within our software solutions, using Python and Langchain.\n• Collaborate with cross-functi

In [72]:
# Step 2: Make messages list

def messages_for(text):
    return [
        {"role": 'system', "content": system_prompt},
        {"role": 'user', "content": user_prompt(text)}
    ]

In [77]:
# Step 3: Call Open AI

def get_tech_skills(text):
    response= openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(text)
    )
    return response.choices[0].message.content

In [83]:
# Step 4: Print the result

get_tech_skills(JD)

'Based on the provided job description for the position of "Python Fullstack LLM Langchain Developer," here are the key technologies and relevant skills highlighted:\n\n### Must-Have Technical Skills\n1. **Programming Languages:**\n   - Python\n   - Knowledge of additional languages (Java, C#, Javascript) - good to have\n\n2. **AI and Machine Learning Technologies:**\n   - Advanced AI/ML model development and deployment\n   - Natural Language Processing (NLP) algorithms\n\n3. **Frameworks and Libraries:**\n   - Langchain\n   - Relevant frameworks for API development (specific frameworks not mentioned)\n\n4. **Database and API Integration:**\n   - Experience integrating with various databases\n   - Experience with third-party APIs\n\n5. **Development Practices:**\n   - Code quality, sustainability, and scalability best practices\n   - Participation in code reviews\n\n6. **CI/CD Tools:**\n   - Kubernetes for automated deployments and testing\n\n7. **Cloud Platforms and AI Services:**\n  

In [79]:
def get_tech_skills_refined(text):
    summary=get_tech_skills(text)
    display(Markdown(summary))

In [84]:
get_tech_skills_refined(JD)

Based on the job description provided, the key technologies and technical skills required for the Python Fullstack LLM Langchain Developer position are:

### Must Have Technologies:
1. **Python** - Core programming language for development.
2. **AI/ML** - Expertise in developing and deploying Artificial Intelligence and Machine Learning models.
3. **NLP** - Proficiency in Natural Language Processing algorithms.
4. **Langchain** - Specific framework used for software solutions involving LLM (Large Language Models).
5. **Back-end API Development** - Experience in designing and maintaining robust and scalable APIs, likely using relevant Python frameworks.
6. **Databases** - Knowledge in integrating various databases and third-party APIs.
7. **CI/CD Pipelines** - Familiarity with continuous integration and continuous deployment practices.
8. **Kubernetes** - Experience with orchestration for automated deployments and testing.

### Exposure to Tools/Technologies:
1. **GCP** (Google Cloud Platform) - Specifically AI services (such as Vertex AI).
2. **Azure** - Familiarity with Azure AI services.
3. **AWS** - Exposure to AWS AI services.
4. **MLOps** - Understanding of practices for machine learning operations.
5. **NLP Techniques** - Keeping updated with the latest in Natural Language Processing field.

### Good to Have Technologies:
1. **Additional Programming Languages** - Knowledge of other programming languages such as Java, C#, or JavaScript is beneficial.

### Preferred Background:
- A degree in Computer Science, Artificial Intelligence, or a related field.
- A portfolio demonstrating practical experience in AI/ML and NLP projects.

This outlines the primary technical skills and technologies that the candidate needs to be familiar with for the position.